In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [2]:
def create_lstm_model(input_shape, output_shape):
    model = Sequential([
        LSTM(50, activation='relu', return_sequences=True, input_shape=input_shape),
        LSTM(50, activation='relu'),
        Dense(output_shape)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

In [3]:
def generate_synthetic_data_glucose(num_individuals=12, num_samples=1000, timesteps=24):
    X_data, y_data, individuals = [], [], []
    for i in range(num_individuals):
        base_value = np.random.uniform(80, 180)  # Define um ponto de partida realista
        noise = np.cumsum(np.random.normal(0, 5, size=(num_samples, timesteps,1)))  # Suaviza variações
        # X = np.clip(base_value + noise, 40, 300)  # Mantém os valores no intervalo desejado
        X = np.clip(base_value + np.random.normal(0, 5, size=(num_samples,timesteps, 1)), 40, 300)
        y = np.clip(base_value + np.random.normal(0, 5, size=(num_samples, 6)), 40, 300)  # Glicose futura
        X_data.append(X)
        y_data.append(y)
        individuals.extend([i] * num_samples)
    return np.vstack(X_data), np.vstack(y_data), np.array(individuals)


In [4]:
def generate_synthetic_data(num_individuals=12, num_samples=1000, timesteps=10):
    X_data, y_data, individuals = [], [], []
    for i in range(num_individuals):
        X = np.random.rand(num_samples, timesteps, 1)  # Sequências de glicose simuladas
        y = np.random.rand(num_samples, 1)  # Valores de glicose futura
        X_data.append(X)
        y_data.append(y)
        individuals.extend([i] * num_samples)
    return np.vstack(X_data), np.vstack(y_data), np.array(individuals)

In [5]:
# Gerando dados sintéticos
X, y, individuals = generate_synthetic_data()
X_gl, y_gl, individuals_gl = generate_synthetic_data_glucose()

In [6]:
print(len(X_gl))
print(len(y_gl))
print(X_gl[0])
print(y_gl[0])
print(individuals_gl[999])
print(individuals_gl[1000])

12000
12000
[[103.00087166]
 [ 92.88345207]
 [100.16792323]
 [ 96.925653  ]
 [103.36958161]
 [ 94.76669535]
 [100.94423116]
 [102.62102675]
 [106.13334347]
 [102.9148513 ]
 [102.82444095]
 [104.22290653]
 [102.37986577]
 [105.25878611]
 [102.89722011]
 [109.21936188]
 [100.93686544]
 [ 93.63719751]
 [109.5647755 ]
 [ 94.61965772]
 [108.20895251]
 [109.11798959]
 [102.68425442]
 [ 96.37743199]]
[107.32593765 104.38048737 106.08543943  98.19261245  99.35973553
 105.55587573]
0
1


In [15]:
X_gl.shape



(12000, 24, 1)

In [7]:
print(len(X))
print(len(y))
print(X[0])
print(y[0])
print(individuals[999])
print(individuals[1000])

12000
12000
[[0.03892328]
 [0.67724476]
 [0.20924335]
 [0.77096238]
 [0.0292669 ]
 [0.75698441]
 [0.30942785]
 [0.7717496 ]
 [0.11957058]
 [0.20272422]]
[0.89577222]
0
1


In [8]:
X.shape[1]

10

In [9]:
X.shape[2]

1

In [18]:
y.shape

(12000, 1)

In [20]:
y_gl.shape

(12000, 6)

In [ ]:
X_gl.shape

(12000, 24, 1)

In [ ]:
# Population Training
X_train, X_test, y_train, y_test = train_test_split(X_gl, y_gl, test_size=0.2, random_state=42, shuffle=False)

model_pop = create_lstm_model(input_shape=(X_gl.shape[1], X_gl.shape[2]),output_shape=y_gl.shape[1])
model_pop.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
y_pred_pop = model_pop.predict(X_test)
rmse_pop = np.sqrt(mean_squared_error(y_test, y_pred_pop))
print(f'Population Training RMSE: {rmse_pop:.4f}')

Epoch 1/10


2025-04-01 11:12:46.204986: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


300/300 [==============================] - 4s 11ms/step - loss: 3224.7805 - val_loss: 134.5418
Epoch 2/10
300/300 [==============================] - 3s 10ms/step - loss: 51.1808 - val_loss: 43.4130
Epoch 3/10
300/300 [==============================] - 3s 10ms/step - loss: 36.6822 - val_loss: 36.6498
Epoch 4/10
300/300 [==============================] - 3s 10ms/step - loss: 33.5439 - val_loss: 33.7268
Epoch 5/10
300/300 [==============================] - 3s 10ms/step - loss: 31.8801 - val_loss: 32.1497
Epoch 6/10
300/300 [==============================] - 3s 10ms/step - loss: 34.5579 - val_loss: 32.4881
Epoch 7/10
300/300 [==============================] - 3s 10ms/step - loss: 37.7235 - val_loss: 85.6711
Epoch 8/10
300/300 [==============================] - 3s 10ms/step - loss: 42.7043 - val_loss: 41.8479
Epoch 9/10
300/300 [==============================] - 3s 10ms/step - loss: 35.2431 - val_loss: 36.1676
Epoch 10/10
75/75 [==============================] - 0s 3ms/step
Population Train

In [ ]:

# Personalized Training
models_personalized = {}
for person_id in np.unique(individuals):
    X_person = X_gl[individuals == person_id]
    y_person = y_gl[individuals == person_id]
    X_train_p, X_test_p, y_train_p, y_test_p = train_test_split(X_person, y_person, test_size=0.2, random_state=42,shuffle=False)
    model = create_lstm_model(input_shape=(X_gl.shape[1], X_gl.shape[2]), output_shape=y_gl.shape[1])
    model.fit(X_train_p, y_train_p, epochs=10, batch_size=8, validation_data=(X_test_p, y_test_p))
    y_pred_p = model.predict(X_test_p)
    rmse_p = np.sqrt(mean_squared_error(y_test_p, y_pred_p))
    print(f'Personalized Training RMSE for individual {person_id}: {rmse_p:.4f}')
    models_personalized[person_id] = model


Epoch 1/10
100/100 [==============================] - 2s 9ms/step - loss: 25273.0527 - val_loss: 24985.3379
Epoch 2/10
100/100 [==============================] - 1s 7ms/step - loss: 58392.2617 - val_loss: 578.6183
Epoch 3/10
100/100 [==============================] - 1s 7ms/step - loss: 197.2065 - val_loss: 150.4345
Epoch 4/10
100/100 [==============================] - 1s 8ms/step - loss: 124.6428 - val_loss: 125.5824
Epoch 5/10
100/100 [==============================] - 1s 7ms/step - loss: 183.9135 - val_loss: 392.1653
Epoch 6/10
100/100 [==============================] - 1s 7ms/step - loss: 260.6801 - val_loss: 222.5048
Epoch 7/10
100/100 [==============================] - 1s 9ms/step - loss: 159.1070 - val_loss: 100.9145
Epoch 8/10
100/100 [==============================] - 1s 8ms/step - loss: 76.4723 - val_loss: 62.6549
Epoch 9/10
100/100 [==============================] - 1s 7ms/step - loss: 63.2709 - val_loss: 56.0910
Epoch 10/10
7/7 [==============================] - 0s 4ms/step

In [13]:
# Fine-Tuning Training
models_finetuned = {}
for person_id in np.unique(individuals):
    X_person = X_gl[individuals == person_id]
    y_person = y_gl[individuals == person_id]
    X_train_p, X_test_p, y_train_p, y_test_p = train_test_split(X_person, y_person, test_size=0.2, random_state=42,shuffle=False)
    model_finetuned = tf.keras.models.clone_model(model_pop)  # Clone do modelo populacional
    model_finetuned.set_weights(model_pop.get_weights())  # Inicializa com pesos treinados na população
    model_finetuned.compile(optimizer='adam', loss='mse')
    model_finetuned.fit(X_train_p, y_train_p, epochs=5, batch_size=8, validation_data=(X_test_p, y_test_p))  # Treino rápido
    y_pred_ft = model_finetuned.predict(X_test_p)
    rmse_ft = np.sqrt(mean_squared_error(y_test_p, y_pred_ft))
    print(f'Fine-Tuning Training RMSE for individual {person_id}: {rmse_ft:.4f}')
    models_finetuned[person_id] = model_finetuned


Epoch 1/5
100/100 [==============================] - 2s 9ms/step - loss: 124.0007 - val_loss: 143.5974
Epoch 2/5
100/100 [==============================] - 1s 8ms/step - loss: 130.1860 - val_loss: 33.3975
Epoch 3/5
100/100 [==============================] - 1s 7ms/step - loss: 34.7266 - val_loss: 33.8589
Epoch 4/5
100/100 [==============================] - 1s 8ms/step - loss: 32.2500 - val_loss: 30.7527
Epoch 5/5
7/7 [==============================] - 0s 4ms/step
Fine-Tuning Training RMSE for individual 0: 5.5330
Epoch 1/5
100/100 [==============================] - 2s 10ms/step - loss: 53.9254 - val_loss: 25.9474
Epoch 2/5
100/100 [==============================] - 1s 7ms/step - loss: 30.2471 - val_loss: 26.2777
Epoch 3/5
100/100 [==============================] - 1s 8ms/step - loss: 29.3814 - val_loss: 31.7944
Epoch 4/5
100/100 [==============================] - 1s 7ms/step - loss: 30.3138 - val_loss: 26.1211
Epoch 5/5
7/7 [==============================] - 0s 4ms/step
Fine-Tuning Tra